Let's get started. First, we import some libraries, we might need an then load the dataset.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
# Set theme
sns.set_style('darkgrid') # 'whitegrid'

In [ ]:
fd = pd.read_csv('falldeteciton.csv')

In [ ]:
fd.head()

After taking a first look at the data I decided to replace the numbers in the ACTIVITY column with the corresponding strings. This was purely aesthetic decision and has no analytical impact. This was it's easier for me to see right away what kind of movement there is in each line. Furthermore, some of the diagrams that follow later are easier to read.

In [ ]:
fd = fd.replace({'ACTIVITY':{0:'Standing',1:'Walking',2:'Sitting',3:'Falling',4:'Cramps',5:'Running'}})
fd.head()

## Descriptive statistics

Take a futher look at the data and inspecting distributions and looking for missings.

In [ ]:
fd.shape

In [ ]:
fd.describe()

In [ ]:
fd.isna().sum()

No Missing Values, so nothing to do here, yay!

OK, let's draw some charts.

In [ ]:
cols=['TIME','SL','EEG','BP','HR','CIRCLUATION']
#Distribution
fig = plt.figure(figsize=(10, 20)) # (Breite,Länge)
for i in range (0,len(cols)):
    fig.add_subplot(len(cols),1,i+1)
    sns.distplot(fd[cols[i]]);

In [ ]:
sns.boxplot(data=fd)

Extreme Ausreisser bei EEG. Schneide unterstes und oberstes Promille von Verteilung ab
Looks like we have some heavy outliers in the EEG column. But it's only a few values, so we can cut small parts of the distribution to remove them.

In [ ]:
fd = fd[(fd['EEG']< fd['EEG'].quantile(0.999) ) & (fd['EEG']> fd['EEG'].quantile(0.001))]

SL also has outliers, so we cut the lowest and highest percent of the distribution

In [ ]:
fd = fd[(fd['SL']< fd['SL'].quantile(0.99) ) & (fd['SL']> fd['SL'].quantile(0.01))]

In [ ]:
sns.boxplot(data=fd)

In [ ]:
sns.pairplot(fd, hue='ACTIVITY', h=2.5);

The scatterplots look very nice and promising. I guess, we can start some modeling.

## Machine Learning

In [ ]:
from sklearn.model_selection import train_test_split
# Classifiier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

Splitting the data in Features and Target and create training and test sets.

In [ ]:
Target = fd['ACTIVITY']
Features = fd[['TIME','SL','EEG','BP','HR','CIRCLUATION']]

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(Features, Target, test_size = 0.3)

Test some classifiers:

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy

In [ ]:
rfc = RandomForestClassifier(
    n_estimators = 50,
    #min_samples_leaf=10,
    #min_samples_split=10
)
rfc.fit(X_train, y_train)
predictions = rfc.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
predictions = knn.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy

In [ ]:
svc = SVC(gamma='auto')
svc.fit(X_train,y_train)
predictions = svc.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy